In [ ]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head()

In [ ]:
def date_sorter():
    import pandas as pd
    import numpy as np
    import re
    doc = []
    with open('dates.txt') as file:
        for line in file:
                doc.append(line)
            
    df = pd.Series(doc)
    df.shape
    
    # Group the extracted dates according to the assignment rules
    # First extraction group is meant to extract 04/20/2009; 04/20/09; 4/20/09; 4/3/09
    g1_1 =df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
    g1_2 =df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
    g1 = pd.concat([g1_1,g1_2])
    g1.reset_index(inplace=True)
    g1_index = g1['level_0']
    
    # Second extraction group is meant to extract dates with monthly names
    # With the following code we can extract both Mar-20-2009 and Mar 20th, 2009 format
    g2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    g2.reset_index(inplace=True)
    g2_index = g2['level_0']
    
    # Here the following code will give us the 20 March, 2009 and Feb 2009 format
    g3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    g3.reset_index(inplace=True)
    g3_index = g3['level_0']
    
    # Last two regular expression should yiled the 6/2008 and 2009 year format
    # First 6/2008 format can be extracted with the following code
    g4 = df.str.extractall(r'(\d{1,2})[/](\d{4})')
    g4.reset_index(inplace=True)
    g4_index = g4['level_0']
    save=[]
    for i in g4_index:
        if not(i in g1_index.values):
            save.append(i)
    save = np.asarray(save)
    g4 = g4[g4['level_0'].isin(save)]

    # Last but not least this code can be used to extract the date in the format of yyyy: 2009;2010
    g5_1= df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    g5_2 = df.str.extractall(r'^(\d{4})[^0-9]')
    g5 = pd.concat([g5_1, g5_2])
    g5.reset_index(inplace=True)

    g5_index = g5['level_0']
    save=[]
    # To obtain the index values that are not either in g2, g3 or in g4 indices: remove the intersection indices
    for i in g5_index:
        if not((i in g2_index.values) | (i in g3_index.values) | (i in g4_index.values)):
            save.append(i)
    save = np.asarray(save)
    g5 = g5[g5['level_0'].isin(save)]
    
    # Second task of the asssignment asks to convert the data into mm/dd/yy format
    # Starting from the first group lets create month, day, and year columns and follow the assignment instructions
    # Assume all dates where year is encoded in only two digits are years from the 1900's
    g1.columns=['level_0','match','month','day','year']
    g1['year']=g1['year'].apply(str)
    g1['year']=g1['year'].apply(lambda x: '19'+x if len(x)<=2 else x)
    
    # Second group dataframe can be grouped in the follwing way: day and year of the date can be decomposed 
    # and be written out as separate columns.
    g2[1] = g2[1].apply(lambda x: x.replace(',',''))
    g2['day'] = g2[1].apply(lambda x:x.split(' ')[0])
    g2['year'] = g2[1].apply(lambda x:x.split(' ')[1])
    g2.columns=['level_0','match','month','day-year','day','year']
    g2.drop('day-year',axis=1,inplace=True) 
    
    # Since some of the rows in group 3 dataframe is encoded as NaN values we can replace them with arbitrary 
    # values and then code them as the first day of the month for the missing values where the day of the dates
    # are represented by NaN values. 
    g3.columns=['level_0','match','day','month','year']
    g3['day'] = g3['day'].replace(np.nan,-99)
    g3['day'] = g3['day'].apply(lambda x: 1 if int(x)==-99 else x)
    
    # Pandas to_datetime method will allow us to turn month names (Apr., Sept.) to its numerical correspondences.
    # Argument format = '%b' is used where %b stands for the abbrevations of the months (i.e. Sep.).
    g3['month'] = g3.month.apply(lambda x: x[:3])
    g3['month'] = pd.to_datetime(g3.month, format='%b').dt.month
    
    # If the day is missing (e.g. 9/2009), assume it is the first day of the month
    g4.columns=['level_0','match','month','year']
    g4['day']=1
    
    # If the month is missing (e.g. 2010), assume it is the first of January of that year 
    g5.columns=['level_0','match','year']
    g5['day']=1
    g5['month']=1
   
    # Finally all of the cleaned data are stacked together and were put into the mm/dd/yy format as assignment task requires.
    final_df = pd.concat([g1,g2,g3,g4,g5])
    final_df['date'] = pd.to_datetime(final_df['month'].apply(str)+'/'+final_df['day'].apply(str)+'/'+final_df['year'].apply(str))
    final_df = final_df.sort_values(by='level_0').set_index('level_0')
    
    # Final step consists of returning a pandas Series in chronological order of the original Series' indices
    my_list = final_df['date']
    ans = pd.Series([i[0] for i in sorted(enumerate(my_list), key=lambda x:x[1])],np.arange(500))
    return ans
date_sorter()